# EZyRB Tutorial 3
## Use NNsPOD to help with POD

In this tutorial we show how to set up and use the NNsPOD class in order to make all data align, allowing the use of POD.

To do this we will show a simple example where the data is a moving gaussian wave.

the first step is to import necessary packages

In [14]:
import numpy as np
import matplotlib
import torch
import torch.nn as nn
from ezyrb.nnspod import NNsPOD
from ezyrb import Database
from ezyrb import POD
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

## 1d Data

Now we make the data we will use. We make a simple gaussian function and populate the space, snapshots, and parameters of the database

In [2]:
n_params = 15
params = np.linspace(0.5, 4.5, n_params).reshape(-1, 1) # actually the time steps
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
def wave(t, res=256):
    x = np.linspace(0, 5, res)
    return  x, gaussian(x, t, 0.1)

db = np.array([wave(t)[1] for t in params])
db_array =  np.array([wave(t)[1] for t in params])
space = wave(0)[0]
space_array = np.array([wave(t)[0] for t in params])

database = Database(space = space_array, snapshots = db_array, parameters = params)


Here we make a NNsPOD class, the only value to pass in is where you want to save the interpnet, or where you want to load it from. This is especially usefull with 2d data where training can take hours depending on the size of the dataset.

In [3]:
NNsPOD_tutorial = NNsPOD(path = "interpnet1d.pth")

Now we train the interpnet. the data to pass in is the reference database, the shape of the layers of the NN, the trainnig function the stop training value, which can be a float(loss value to stop at), int(epoch to stop at), or both(will stop at whichever is reached first).The loss function(MSE by default). and whether you would like to retrain NN or load a saved NN. If you choose to retrain the loss value at each epoch will be printed out.

In [4]:
ref_data = 5
NNsPOD_tutorial.train_interpnet(database[ref_data], [20,20], nn.Sigmoid(), [0.000001], None, retrain  = False, frequency_print = 5)

loaded interpnet


Now we can graph the original reference data as well as the data we get from the interpNet after feeding it 1000 positional datapoints. The large points are the original data points, and the small points are ones created by the interpnet. It should be clear that the interpnet is able to accuratly replicate the gaussian

In [5]:
plt.plot(database[ref_data].space, database[ref_data].snapshots, "o")
xi = np.linspace(0,5,1000).reshape(-1,1)
yi = NNsPOD_tutorial.interp_net.predict(xi)
plt.plot(xi,yi, ".")
plt.show()

Now we train the shiftnet on all data besides the reference. to do this you must pass in the database at the value, the shape of the NN, the training function, the stop training value, the reference database, and if you would like the data to be preshifted. For the shiftnet it can be useful to put a loss value and epoch value to stop at, as there is a minimum level the loss value can reach, and if you put a lower value the neural net will not stop.

Training the shiftnet also prints out the loss value at every epoch

In [6]:
i = 0
while i < 10:
    x_new  = NNsPOD_tutorial.train_shiftnet(database[i], [20,20,20], nn.Tanh(), [10000, 0.00001], database[ref_data], preshift = True, frequency_print = 5, learning_rate = 0.0001) 
    db = database[i]   
    plt.plot(db.space, db.snapshots, "go")
    plt.plot(x_new, db.snapshots.reshape(-1,1), ".")
    i+=1
    if i == ref_data:
        i +=1

0.0038859164342284203
0.0015381069388240576
0.00022182625252753496
0.0016359166475012898
0.0006663238164037466
0.00011276069562882185
0.06215149909257889
0.06021205335855484
0.05788048729300499
0.05500495433807373
0.0514841228723526
0.04724118486046791
0.04224418103694916
0.0365261510014534
0.03020881861448288
0.02352849766612053
0.01685439981520176
0.010667995549738407
0.0055058738216757774
0.001872258959338069
0.00013740750728175044
0.00044998101657256484
0.0026935155037790537
0.006505591329187155
0.011355931870639324
0.016656838357448578
0.021869273856282234
0.026576608419418335
0.030514836311340332
0.03355848044157028
0.03567685931921005
0.03688859939575195
0.03722931072115898
0.036733899265527725
0.03543071448802948
0.033345889300107956
0.0305155161768198
0.02700406312942505
0.02292546257376671
0.01846104860305786
0.013866759836673737
0.00946125853806734
0.005591918248683214
0.002582910005003214
0.0006791849154978991
3.463618668320123e-06
0.045038238167762756
0.04062008112668991
0

Here we plot and show all the data. The original positions is represented by green circles, the reference data is the blue plusmarks, and the different shifted data is represented by the smaller dots. It should be clear that all of the data has been moved to allign with the reference data

In [7]:
plt.plot(database[0].space, database[ref_data].snapshots, "b+")
plt.show()

Here we use the NNsPOD fit function, and we get get the modes and singular values

In [8]:
pod = NNsPOD_tutorial.fit(database, 5, interp_loss = [0.000001], interp_function = nn.Sigmoid(), interp_layers = [20,20],
                            shift_loss = [0.00001], shift_function = nn.Tanh(), shift_layers = [20,20,20])
print(pod.modes, pod.singular_values)

loaded interpnet
0.06858384609222412
0.06757062673568726
0.06625917553901672
0.0645105317234993
0.06218956038355827
0.05915028974413872
0.05524875596165657
0.05036800354719162
0.04446140304207802
0.03760220482945442
0.03003794513642788
0.02220197208225727
0.014689859002828598
0.008188384585082531
0.0033462457358837128
0.0006332324119284749
0.00023231202794704586
0.0019962091464549303
0.005485348869115114
0.010075119324028492
0.01509601529687643
0.019968131557106972
0.024286048486828804
0.027823377400636673
0.03048844076693058
0.03226960077881813
0.03318987786769867
0.03327736258506775
0.03255076706409454
0.03101884201169014
0.028690703213214874
0.02559574693441391
0.021811433136463165
0.01749490760266781
0.012909196317195892
0.008430377580225468
0.004519726615399122
0.0016545085236430168
0.0002288002142449841
0.0004562921531032771
0.0023141324054449797
0.00555219454690814
0.00976277980953455
0.014479910023510456
0.01927080564200878
0.023794325068593025
0.027821017429232597
0.0312240310

In [16]:
n_params = 15
params = np.linspace(0.5, 4.5, n_params).reshape(-1, 1) # actually the time steps
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
def wave(t, res=256):
    x = np.linspace(0, 5, res)
    return  x, gaussian(x, 2, 0.1)

db = np.array([wave(t)[1] for t in params])
db_array =  np.array([wave(t)[1] for t in params])
space = wave(0)[0]
space_array = np.array([wave(t)[0] for t in params])

database = Database(space = space_array, snapshots = db_array, parameters = params)

pod = POD()

pod.fit(database.snapshots)
print(pod.modes, pod.singular_values)



[[-2.58198890e-01 -1.41311923e-01  9.55700933e-01 -5.76213568e-16
   2.05305470e-17  3.92872887e-17  1.30957629e-16 -4.36525430e-17
  -8.73050861e-18  0.00000000e+00  8.73050861e-18  0.00000000e+00
   0.00000000e+00 -8.73050861e-17 -8.73050861e-17]
 [-2.58198890e-01 -9.78417094e-02 -8.42239073e-02  9.57427108e-01
   1.94065217e-17 -5.78557981e-17 -7.56721665e-17 -9.21162325e-18
   1.77285449e-17  5.41821909e-18 -7.53123300e-18  4.37232097e-19
  -3.66929070e-18  1.04239796e-17 -8.11468274e-17]
 [-2.58198890e-01 -9.78417094e-02 -8.42239073e-02 -8.70388280e-02
  -1.68819141e-18  2.54770892e-01  7.18048562e-01 -6.05963946e-02
   1.45995894e-16  7.66610858e-17 -5.29868291e-17  1.91153105e-17
  -1.99843747e-19 -4.03061435e-01 -4.03061435e-01]
 [-2.58198890e-01 -9.78417094e-02 -8.42239073e-02 -8.70388280e-02
   3.62069297e-18  1.63575481e-01  4.61022600e-01 -3.89058747e-02
   6.10372215e-17  4.72135646e-17 -1.78693732e-17  1.17534302e-17
  -1.99843747e-19  5.78047777e-01  5.78047777e-01]
 [-2

## 2d Data

Now we do the same but with 2d data and implement some basic functions to help with shaping the data

In [9]:
def reshape2dto1d(x, y):
    x = x.reshape(-1,1)
    y = y.reshape(-1,1)
    coords = np.concatenate((x, y), axis = 1)
    coords = np.array(coords).reshape(-1,2)
    
    return coords

def reshape1dto2d(snapshots):
        return snapshots.reshape(int(np.sqrt(len(snapshots))), int(np.sqrt(len(snapshots))))


Here we create the 2d gaussian and populate the database

In [10]:

n_params = 10
params = np.linspace(0.5, 4.5, n_params).reshape(-1, 1) # actually the time steps

def gaussian(x, mu, sig):
    print(mu)
    gaussx, gaussy = np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))).T
    return gaussx * gaussy
def wave(t, res=256):
    x = np.linspace(0, 5, res)
    return  x, gaussian(x, t, 0.1)
def wave2D(t, res=256):
    x = np.linspace(0, 5, res)
    y = np.linspace(0, 5, res)
    gridx, gridy = np.meshgrid(x, y)
    gridx, gridy = gridx.reshape(-1,1), gridy.reshape(-1,1)
    wave = gaussian(np.hstack([gridx, gridy]), t*np.array([1, 1]), 0.1)
    return  gridx, gridy, wave
db = np.array([wave2D(t)[2] for t in params])
db_array = db.reshape(n_params, -1, 1)
gridx, gridy = wave2D(0)[0 :2]
space = reshape2dto1d(gridx,gridy)
space_array = np.array([space.copy() for t in params])

database = Database(space = space_array, snapshots = db_array, parameters = params)


[0.5 0.5]
[0.94444444 0.94444444]
[1.38888889 1.38888889]
[1.83333333 1.83333333]
[2.27777778 2.27777778]
[2.72222222 2.72222222]
[3.16666667 3.16666667]
[3.61111111 3.61111111]
[4.05555556 4.05555556]
[4.5 4.5]
[0 0]


In [11]:
NNsPOD_tutorial = NNsPOD(path = "interpnet2d.pth")

Here we put the value for the reference data and train the interpnet. If you want to change this it will need to retrain the interpnet, which can take a long time for 2d data, especially is the loss value is very low.

In [12]:

ref_data = 5
NNsPOD_tutorial.train_interpnet(database[ref_data], [40,40], nn.Sigmoid(), [10000000,0.000001], None, retrain  = False, frequency_print = 5)

0.016377253457903862
0.0021812450140714645
0.0035201357677578926
0.0013941740617156029
0.0021817893721163273
0.0012498707510530949
0.0015983553603291512
0.0012449437053874135
0.001372409169562161
0.0012416832614690065
0.001288054627366364
0.00124117371160537
0.001254407805390656
0.0012420185375958681
0.0012415293604135513
0.001241300138644874
0.0012382266577333212
0.0012393658980727196
0.0012381538981571794
0.001238068100064993
0.0012381216511130333
0.0012378172250464559
0.0012378423707559705
0.0012378108222037554
0.001237735035829246
0.0012377257226034999
0.00123770406935364
0.001237671822309494
0.0012376517988741398
0.0012376316590234637
0.001237607910297811
0.0012375853257253766
0.0012375636724755168
0.0012375410879030824
0.0012375180376693606
0.001237494871020317
0.0012374714715406299
0.001237447839230299
0.0012374237412586808
0.0012373995268717408
0.0012373749632388353
0.0012373501667752862
0.0012373250210657716
0.0012372996425256133
0.0012372741475701332
0.0012372481869533658
0.0

KeyboardInterrupt: 

Here we graph the reference data

In [ ]:
x = np.linspace(0, 5, 256)
y = np.linspace(0, 5, 256)
gridx, gridy = np.meshgrid(x, y)
        
plt.pcolor(gridx,gridy,database[ref_data].snapshots.reshape(256, 256))
plt.show()

Now we graph the interpolated data. it should be visisble that we get the same function, but with better resolution

In [ ]:
res = 1000
x = np.linspace(0, 5, res)
y = np.linspace(0, 5, res)
gridx, gridy = np.meshgrid(x, y)
input = NNsPOD_tutorial.reshape2dto1d(gridx, gridy)
output = NNsPOD_tutorial.interp_net.predict(input)

toshow = NNsPOD_tutorial.reshape1dto2d(output)
plt.pcolor(gridx,gridy,toshow)
plt.show()

Now we use the shiftnet and graph the shifted data. For each parameter we first graph the refrence data, then the input data, then it will take some time to shift it, and finally it will graph the shifted data. The loss value at every epoch will be printed out as well.

In [ ]:
i = 0
x = np.linspace(0, 5, 256)
y = np.linspace(0, 5, 256)
gridx, gridy = np.meshgrid(x, y)
while i < 10:
    db = database[i]
    plt.pcolor(gridx,gridy,database[ref_data].snapshots.reshape(256, 256))
    plt.show()
    plt.pcolor(gridx,gridy,database[i].snapshots.reshape(256, 256))
    plt.show()
    x_new = NNsPOD_tutorial.train_shiftnet(database[i], [20,20,20], nn.PReLU(), [0.001], database[ref_data], preshift = True, frequency_print = 5)
    x, y = np.hsplit(x_new, 2)
    x = NNsPOD_tutorial.reshape1dto2d(x)
    y = NNsPOD_tutorial.reshape1dto2d(y)
    snapshots = NNsPOD_tutorial.reshape1dto2d(db.snapshots.reshape(-1,1))
    plt.pcolor(x,y,snapshots)
    plt.show()
    res = 256
    i+=1
    if i == ref_data:
        i +=1

